# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

import sys
import os
import re
from sqlalchemy import create_engine
import pickle

from scipy.stats import gmean
# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load data from database
engine = create_engine('sqlite:///beso_disaster_response_db.db')
df = pd.read_sql_table('beso_disaster_response_db',engine)
# a quick check on the min value for each columns
df.min()

id                             2
message                         
genre                     direct
related                        0
request                        0
offer                          0
aid_related                    0
medical_help                   0
medical_products               0
search_and_rescue              0
security                       0
military                       0
child_alone                    0
water                          0
food                           0
shelter                        0
clothing                       0
money                          0
missing_people                 0
refugees                       0
death                          0
other_aid                      0
infrastructure_related         0
transport                      0
buildings                      0
electricity                    0
tools                          0
hospitals                      0
shops                          0
aid_centers                    0
other_infr

In [4]:
# we remove 'child_alone' column since it has only 0  values.
df = df.drop('child_alone', axis = 1)

In [5]:
# maximum value for 'related' column is '2'. Let us investigate further...
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,132,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188


<h2>Note:</h2>

<h5>From the results above, '2' could be error. We can replace it with '0' or '1'. In this case, I will replace it with '1'

</h5>

In [6]:
df['related'] = df['related'].map(lambda x: 1 if x==2 else x) 

In [7]:
# check if all '2' have been replace with '1'
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,6775,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


In [8]:
# separate the dataset to input variables (X) and target variables (y)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    
    # Replaceing all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extracting all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replaceing url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extracting the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [10]:
# Building a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
def pipeline(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

# include StartingVerbExtractor custom transformer
def pipeline_2(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred_test = model.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4984
               request       0.79      0.55      0.65      1134
                 offer       0.00      0.00      0.00        32
           aid_related       0.77      0.60      0.67      2689
          medical_help       0.55      0.27      0.37       513
      medical_products       0.66      0.30      0.41       327
     search_and_rescue       0.64      0.24      0.35       171
              security       0.18      0.04      0.06       102
              military       0.53      0.31      0.39       190
                 water       0.73      0.69      0.71       424
                  food       0.80      0.66      0.72       744
               shelter       0.73      0.50      0.60       550
              clothing       0.79      0.36      0.49       106
                 money       0.53      0.26      0.35       155
        missing_people       0.65      

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
model_2 = pipeline()



parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}


cv = GridSearchCV(model_2, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 



cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estima

[Parallel(n_jobs=-1)]: Done   6 out of  20 | elapsed: 17.5min remaining: 40.7min


[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.22583926754832145, total=13.7min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.24281718789727944, total=13.8min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.24307144673277398, total=14.3min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.22049847405900305, total=14.5min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2433257055682685, total=19.6min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23550356052899288, total=19.8min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.2293997965412004, total=19.8min


[Parallel(n_jobs=-1)]: Done  13 out of  20 | elapsed: 21.6min remaining: 11.7min


[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.24923702950152593, total=19.7min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23671497584541062, total=19.8min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.24364191251271616, total=19.7min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2332146490335707, total=19.8min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.24586829392321383, total=20.1min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.22583926754832145, total=20.1min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.25146198830409355, total=20.1min


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 21.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 21.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
y_pred_test = cv.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      4984
               request       0.77      0.52      0.62      1134
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.54      0.63      2689
          medical_help       0.55      0.27      0.36       513
      medical_products       0.67      0.28      0.40       327
     search_and_rescue       0.70      0.25      0.36       171
              security       0.14      0.03      0.05       102
              military       0.48      0.29      0.36       190
                 water       0.77      0.63      0.70       424
                  food       0.81      0.62      0.70       744
               shelter       0.76      0.49      0.59       550
              clothing       0.84      0.36      0.50       106
                 money       0.57      0.26      0.36       155
        missing_people       0.70      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [30]:
r_model = pipeline(clf = RandomForestClassifier())
r_model.fit(X_train, y_train)
y_pred_r_test = r_model.predict(X_test)

print(classification_report(y_test.values, y_pred_r_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4984
               request       0.85      0.39      0.53      1134
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.53      0.62      2689
          medical_help       0.63      0.08      0.14       513
      medical_products       0.69      0.11      0.19       327
     search_and_rescue       0.80      0.05      0.09       171
              security       1.00      0.01      0.02       102
              military       0.61      0.09      0.16       190
                 water       0.87      0.24      0.38       424
                  food       0.85      0.28      0.42       744
               shelter       0.80      0.23      0.36       550
              clothing       0.80      0.08      0.14       106
                 money       0.86      0.04      0.07       155
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [33]:
with open('classifier.pkl', 'wb') as b:
    pickle.dump(r_model, b)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.